In [43]:
# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import time

# Gráficos
# ==============================================================================
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Otros
# ==============================================================================
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [44]:
# Lectura de la fuente de datos
# ==============================================================================

dataCobranza = pd.read_csv('ContactoCobranza.csv', sep=';')
dataCobranza.head(5)

,MES,CLIENTE,NRO_VEC_COB,PDPs_ROTAS,ESTADO_PDP,NRO_CUOTAS,MES_0,MES_1,MES_2,FECHALLAMADA,HORA,DEUDA_TOTAL,ESTATUS,ACTIVACION,MORA,TIPOCONTACTO
0,201402,C00001,NaN,0,0,"<24, 48]",244.86,0.00,245.26,03/02/2014,15.0,3816.34,BT,2012.0,1.0,COEF
1,201402,C00002,<=10,0,0,>48,610.58,612.69,611.54,18/02/2014,9.0,13620.50,BT,2012.0,1.0,COEF
2,201402,C00003,<=10,>0,0,"<24, 48]",2138.73,0.00,NaN,05/02/2014,16.0,18968.50,BT,2013.0,0.0,CNE
3,201402,C00004,>10,0,0,"<24, 48]",323.07,46.85,74.26,24/02/2014,19.0,2459.84,BT,2012.0,0.0,CNE
4,201402,C00005,<=10,0,0,"<24, 48]",920.54,0.00,0.00,12/02/2014,14.0,6591.76,BT,2013.0,0.0,CNE


In [45]:
# Validación de las columnas que tengan nulos
# ==============================================================================

dataCobranza.isnull().any()

MES             False
CLIENTE         False
NRO_VEC_COB      True
PDPs_ROTAS      False
ESTADO_PDP      False
NRO_CUOTAS       True
MES_0           False
MES_1           False
MES_2            True
FECHALLAMADA    False
HORA            False
DEUDA_TOTAL     False
ESTATUS         False
ACTIVACION      False
MORA            False
TIPOCONTACTO    False
dtype: bool

In [46]:
# Métodos correctivos
# ==============================================================================

dataCobranza["NRO_VEC_COB"] = dataCobranza["NRO_VEC_COB"].fillna('=0')
dataCobranza["NRO_CUOTAS"] = dataCobranza["NRO_CUOTAS"].fillna('=0')
dataCobranza["MES_2"] = dataCobranza["MES_2"].fillna(0)
dataCobranza.head(5)

,MES,CLIENTE,NRO_VEC_COB,PDPs_ROTAS,ESTADO_PDP,NRO_CUOTAS,MES_0,MES_1,MES_2,FECHALLAMADA,HORA,DEUDA_TOTAL,ESTATUS,ACTIVACION,MORA,TIPOCONTACTO
0,201402,C00001,=0,0,0,"<24, 48]",244.86,0.00,245.26,03/02/2014,15.0,3816.34,BT,2012.0,1.0,COEF
1,201402,C00002,<=10,0,0,>48,610.58,612.69,611.54,18/02/2014,9.0,13620.50,BT,2012.0,1.0,COEF
2,201402,C00003,<=10,>0,0,"<24, 48]",2138.73,0.00,0.00,05/02/2014,16.0,18968.50,BT,2013.0,0.0,CNE
3,201402,C00004,>10,0,0,"<24, 48]",323.07,46.85,74.26,24/02/2014,19.0,2459.84,BT,2012.0,0.0,CNE
4,201402,C00005,<=10,0,0,"<24, 48]",920.54,0.00,0.00,12/02/2014,14.0,6591.76,BT,2013.0,0.0,CNE


In [47]:
# Verificar tipo de datos
# ==============================================================================

dataCobranza.dtypes

MES               int64
CLIENTE          object
NRO_VEC_COB      object
PDPs_ROTAS       object
ESTADO_PDP        int64
NRO_CUOTAS       object
MES_0           float64
MES_1           float64
MES_2           float64
FECHALLAMADA     object
HORA            float64
DEUDA_TOTAL     float64
ESTATUS          object
ACTIVACION      float64
MORA            float64
TIPOCONTACTO     object
dtype: object

In [48]:
# Corregir tipo de datos
# ==============================================================================

dataCobranza['MES'] = dataCobranza['MES'].astype('object')
dataCobranza['FECHALLAMADA'] = pd.to_datetime(dataCobranza['FECHALLAMADA'])
dataCobranza['HORA'] = dataCobranza['HORA'].astype('int64')
dataCobranza['ACTIVACION'] = dataCobranza['ACTIVACION'].astype('object')
dataCobranza['MORA'] = dataCobranza['MORA'].astype('int64')
dataCobranza.dtypes

MES                     object
CLIENTE                 object
NRO_VEC_COB             object
PDPs_ROTAS              object
ESTADO_PDP               int64
NRO_CUOTAS              object
MES_0                  float64
MES_1                  float64
MES_2                  float64
FECHALLAMADA    datetime64[ns]
HORA                     int64
DEUDA_TOTAL            float64
ESTATUS                 object
ACTIVACION              object
MORA                     int64
TIPOCONTACTO            object
dtype: object

In [81]:
# Se adiciona campo del RANGO HORARIO
# ==============================================================================

conditions_range = [
    (dataCobranza['HORA'] >= 0) & (dataCobranza['HORA'] < 6),
    (dataCobranza['HORA'] >= 6) & (dataCobranza['HORA'] < 12),
    (dataCobranza['HORA'] >= 12) & (dataCobranza['HORA'] < 18),
    (dataCobranza['HORA'] >= 18)
    ]
values_range = ['MADRUGADA', 'MAÑANA', 'TARDE', 'NOCHE']
values_range2 = [0, 1, 2, 3]
dataCobranza['RANGO'] = np.select(conditions_range, values_range)
dataCobranza['N_RANGO'] = np.select(conditions_range, values_range2)
dataCobranza.head(5)

,MES,CLIENTE,NRO_VEC_COB,PDPs_ROTAS,ESTADO_PDP,NRO_CUOTAS,MES_0,MES_1,MES_2,FECHALLAMADA,...,DEUDA_TOTAL,ESTATUS,ACTIVACION,MORA,TIPOCONTACTO,RANGO,N_MES,N_DIA,DIA,N_RANGO
0,March,C00001,=0,0,0,"<24, 48]",244.86,0.00,245.26,2014-03-02,...,3816.34,BT,2012.0,1,COEF,TARDE,3,6,Sunday,2
1,February,C00002,<=10,0,0,>48,610.58,612.69,611.54,2014-02-18,...,13620.50,BT,2012.0,1,COEF,MAÑANA,2,1,Tuesday,1
2,May,C00003,<=10,>0,0,"<24, 48]",2138.73,0.00,0.00,2014-05-02,...,18968.50,BT,2013.0,0,CNE,TARDE,5,4,Friday,2
3,February,C00004,>10,0,0,"<24, 48]",323.07,46.85,74.26,2014-02-24,...,2459.84,BT,2012.0,0,CNE,NOCHE,2,0,Monday,3
4,December,C00005,<=10,0,0,"<24, 48]",920.54,0.00,0.00,2014-12-02,...,6591.76,BT,2013.0,0,CNE,TARDE,12,1,Tuesday,2


In [106]:
# Se adiciona campos calculados paa el análisis descriptivo
# ==============================================================================

dataCobranza['N_MES'] = pd.DatetimeIndex(dataCobranza['FECHALLAMADA']).month
dataCobranza['MES'] = pd.DatetimeIndex(dataCobranza['FECHALLAMADA']).month_name()
dataCobranza['N_DIA'] = pd.DatetimeIndex(dataCobranza['FECHALLAMADA']).dayofweek
dataCobranza['DIA'] = pd.DatetimeIndex(dataCobranza['FECHALLAMADA']).day_name()
dataCobranza["PERIODO"] = pd.to_datetime(dataCobranza["FECHALLAMADA"]).dt.strftime('%Y%m')
dataCobranza.head(5)

,MES,CLIENTE,NRO_VEC_COB,PDPs_ROTAS,ESTADO_PDP,NRO_CUOTAS,MES_0,MES_1,MES_2,FECHALLAMADA,...,ESTATUS,ACTIVACION,MORA,TIPOCONTACTO,RANGO,N_MES,N_DIA,DIA,N_RANGO,PERIODO
0,March,C00001,=0,0,0,"<24, 48]",244.86,0.00,245.26,2014-03-02,...,BT,2012.0,1,COEF,TARDE,3,6,Sunday,2,201403
1,February,C00002,<=10,0,0,>48,610.58,612.69,611.54,2014-02-18,...,BT,2012.0,1,COEF,MAÑANA,2,1,Tuesday,1,201402
2,May,C00003,<=10,>0,0,"<24, 48]",2138.73,0.00,0.00,2014-05-02,...,BT,2013.0,0,CNE,TARDE,5,4,Friday,2,201405
3,February,C00004,>10,0,0,"<24, 48]",323.07,46.85,74.26,2014-02-24,...,BT,2012.0,0,CNE,NOCHE,2,0,Monday,3,201402
4,December,C00005,<=10,0,0,"<24, 48]",920.54,0.00,0.00,2014-12-02,...,BT,2013.0,0,CNE,TARDE,12,1,Tuesday,2,201412


In [83]:
# Análisis por mes del total de llamadas realizadas
# ==============================================================================

dataMes = dataCobranza.groupby(['N_MES','MES']).size().reset_index(name='CANTIDAD')
dataMes.sort_values(by=['N_MES']).head(5)

,N_MES,MES,CANTIDAD
0,1,January,108
1,2,February,1275
2,3,March,2229
3,4,April,1854
4,5,May,393


In [91]:
# Gráfico por mes del total de llamadas realizadas
# ==============================================================================

fig = make_subplots(rows=1, cols=2)

dataMes = dataMes.sort_values('N_MES', ascending=True)

# Gráfico considerando solo los heridos
trace1  = px.bar(dataMes, x="MES", y="CANTIDAD", color="MES", barmode="group")

fig.add_trace(trace1['data'][0], row=1, col=1)
trace1.show()

In [86]:
# Análisis por rango de hora del total de llamadas realizadas
# ==============================================================================

dataRango = dataCobranza.groupby(['N_RANGO','RANGO']).size().reset_index(name='CANTIDAD')
dataRango.sort_values(by=['N_RANGO']).head(5)

,N_RANGO,RANGO,CANTIDAD
0,1,MAÑANA,4266
1,2,TARDE,3240
2,3,NOCHE,952


In [88]:
# Gráfico por rango de hora del total de llamadas realizadas
# ==============================================================================

# Representación gráfica circular del número de llamadas por rango de hora
fig = px.pie(dataRango, values='CANTIDAD', names='RANGO', title='Rango horario de Llamadas realizadas')
fig.show()

In [90]:
# Análisis por día de semana del total de llamadas realizadas
# ==============================================================================

dataDia = dataCobranza.groupby(['N_DIA','DIA']).size().reset_index(name='CANTIDAD')
dataDia.sort_values(by=['N_DIA']).head(5)

,N_DIA,DIA,CANTIDAD
0,0,Monday,1450
1,1,Tuesday,1766
2,2,Wednesday,1341
3,3,Thursday,1576
4,4,Friday,1392


In [95]:
# Gráfico por mes del total de llamadas realizadas
# ==============================================================================

fig = make_subplots(rows=1, cols=2)

dataDia = dataDia.sort_values('N_DIA', ascending=True)

# Gráfico considerando solo los heridos
trace1  = px.bar(dataDia, x="DIA", y="CANTIDAD", color="DIA", barmode="group")

fig.add_trace(trace1['data'][0], row=1, col=1)
trace1.show()

In [97]:
# Análisis por hora del total de llamadas realizadas
# ==============================================================================

dataHora = dataCobranza.groupby(['HORA']).size().reset_index(name='CANTIDAD')
dataHora.sort_values(by=['HORA']).head(5)

,HORA,CANTIDAD
0,7,804
1,8,870
2,9,888
3,10,842
4,11,862


In [99]:
# Gráfico por hora del total de llamadas realizadas
# ==============================================================================

fig = make_subplots(rows=1, cols=2)

dataHora = dataHora.sort_values('HORA', ascending=True)

# Gráfico considerando solo los heridos
trace1  = px.bar(dataHora, x="HORA", y="CANTIDAD", color="HORA", barmode="group")

fig.add_trace(trace1['data'][0], row=1, col=1)
trace1.show()

In [123]:
# Análisis por tipo de contacto del total de llamadas realizadas
# ==============================================================================


conditions_Graph = [
    (dataTContacto['TIPOCONTACTO'] == 'CNE'),
    (dataTContacto['TIPOCONTACTO'] == 'COEF')
    ]

values_Graph = ['Contacto No Efectivo','Contacto Efectivo']

dataTContacto = dataCobranza.groupby(['TIPOCONTACTO']).size().reset_index(name='CANTIDAD')
dataTContacto['DES_TIPOCONTACTO'] = np.select(conditions_Graph, values_Graph)
dataTContacto.sort_values(by=['TIPOCONTACTO']).head(5)

,TIPOCONTACTO,CANTIDAD,DES_TIPOCONTACTO
0,CNE,5974,Contacto No Efectivo
1,COEF,2484,Contacto Efectivo


In [124]:
# Gráfico por tipo de contacto del total de llamadas realizadas
# ==============================================================================

# Representación gráfica circular del número de llamadas por tipo de contacto
fig = px.pie(dataTContacto, values='CANTIDAD', names='DES_TIPOCONTACTO', title='Tipo de contacto de Llamadas realizadas')
fig.show()

In [147]:
# Análisis de la contactibilidad por periodo (según fecha de llamada)
# ==============================================================================
dataContacto_CNE = dataCobranza[dataCobranza['TIPOCONTACTO'] == 'CNE'].groupby(['PERIODO']).size().reset_index(name='CNE')
dataContacto_CNE.sort_values(by=['PERIODO']).head(5)

dataContacto_COEF = dataCobranza[dataCobranza['TIPOCONTACTO'] == 'COEF'].groupby(['PERIODO']).size().reset_index(name='COEF')
dataContacto_COEF.sort_values(by=['PERIODO']).head(5)

dataContacto = pd.merge(dataContacto_CNE, dataContacto_COEF, how='outer', on = 'PERIODO')
dataContacto["CONTACTABILIDAD"] = (dataContacto["COEF"] / (dataContacto["COEF"]+dataContacto["CNE"]))
dataContacto["CONT_PORC"] = round(dataContacto["CONTACTABILIDAD"]*100,2)
dataContacto

,PERIODO,CNE,COEF,CONTACTABILIDAD,CONT_PORC
0,201401,89,19,0.175926,17.59
1,201402,943,332,0.260392,26.04
2,201403,1556,673,0.301929,30.19
3,201404,1299,555,0.299353,29.94
4,201405,284,109,0.277354,27.74
5,201406,295,155,0.344444,34.44
6,201407,362,143,0.283168,28.32
7,201408,116,57,0.329480,32.95
8,201409,108,56,0.341463,34.15
9,201410,345,153,0.307229,30.72


In [153]:
dataContactoGraph = dataContacto.melt(id_vars=['PERIODO'], 
                             var_name="TIPO", 
                             value_name="LLAMADAS")

#dataContactoGraph
fig = px.bar(dataContactoGraph.query("TIPO!='CONTACTABILIDAD' & TIPO!='CONT_PORC'"), x="PERIODO", y="LLAMADAS", 
                                     color="TIPO", text="LLAMADAS", title='Cantidad de llamadas por tipo de contacto')
fig2 = px.line(dataContacto, x="PERIODO", y="CONT_PORC", title='% Contactabilidad por periodo', text="CONT_PORC")
fig2.show()
fig.show()